In [ ]:
%pip install numpy
%pip install pandas
%pip install nltk
%pip install scikit-learn
%pip install re 

In [2]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import ssl

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

In [5]:
dataset = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
dataset.head()

In [ ]:
col_names = ['target', 'id', 'date', 'flag', 'user', 'text']
dataset.columns = col_names
dataset.head()

In [8]:
dataset.shape

(1599999, 6)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset['target'] = dataset['target'].map({0: 0, 4: 1})
dataset['target'].value_counts()

In [11]:
stremmer = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stremmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [12]:
dataset['text'] = dataset['text'].apply(stemming)

In [ ]:
dataset.head()

In [14]:
x = dataset['text']
y = dataset['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [15]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)
print(accuracy_score(y_test, y_pred))


In [22]:
def predict_sentiment(text):
    text = stemming(text)
    text = [text]
    text = vectorizer.transform(text)
    sentiment = model.predict(text)
    if sentiment == 0:
        return 'Negative'
    else:
        return 'Positive'

In [ ]:
print(predict_sentiment("I love this product!"))
print(predict_sentiment("I hate this product!"))

In [24]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [25]:
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))